In [1]:
!pip install transformers==4.39.3 tf-keras==2.14.1 datasets torch scikit-learn pandas


In [13]:
import sys
!{sys.executable} -m pip install accelerate==0.28.0


  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.7.0
    Uninstalling accelerate-1.7.0:
      Successfully uninstalled accelerate-1.7.0


In [11]:
import sys
!{sys.executable} -m pip install accelerate --upgrade


In [1]:

from transformers import (
    BartTokenizer,
    BartForConditionalGeneration,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments
)
import pandas as pd
from datasets import Dataset
import torch
from sklearn.model_selection import train_test_split



/opt/anaconda3/envs/rad-nlp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
W0617 10:58:02.141000 4798 site-packages/torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


In [3]:
# Load your CSV file (use correct path if needed)
df = pd.read_csv("synthetic_radiology_reports.csv")

# Split into training and testing datasets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Convert to Hugging Face Dataset format
train_dataset = Dataset.from_pandas(train_df[['report', 'summary']])
test_dataset = Dataset.from_pandas(test_df[['report', 'summary']])


In [5]:
# Load tokenizer and model
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

# Tokenization function (uses text_target to avoid deprecation warning)
def tokenize_function(example):
    model_inputs = tokenizer(example['report'], max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(text_target=example['summary'], max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply tokenization
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)


/opt/anaconda3/envs/rad-nlp/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Map: 100%|████████████████████████| 1000/1000 [00:00<00:00, 11354.65 examples/s]


In [7]:
from transformers import BartTokenizer, BartForConditionalGeneration

# Load tokenizer and BART model
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')


/opt/anaconda3/envs/rad-nlp/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [9]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./rad_summarizer_model",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=1,
    logging_dir=None,
    logging_strategy="no",
    save_strategy="no",
    report_to="none"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer
)


/opt/anaconda3/envs/rad-nlp/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [11]:
trainer.train()


/opt/anaconda3/envs/rad-nlp/lib/python3.10/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss


TrainOutput(global_step=2000, training_loss=0.06272219467163086, metrics={'train_runtime': 2152.5978, 'train_samples_per_second': 1.858, 'train_steps_per_second': 0.929, 'total_flos': 4334209204224000.0, 'train_loss': 0.06272219467163086, 'epoch': 1.0})

In [15]:
# Force CPU use to avoid MPS embedding issue
device = torch.device("cpu")
model = model.to(device)

# Re-prepare input tensors on CPU
inputs = tokenizer(sample_texts, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)

# Generate summaries
model.eval()
with torch.no_grad():
    generated_ids = model.generate(inputs["input_ids"], max_length=100, num_beams=4)

# Decode summaries
generated_summaries = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

# Show comparison
for i in range(5):
    print(f"\n🔹 Report {i+1}:\n{sample_texts[i]}")
    print(f"✅ Actual Summary:     {true_summaries[i]}")
    print(f"🤖 Generated Summary:  {generated_summaries[i]}")



🔹 Report 1:
Clear lungs. opacity present in the left upper lobe.
✅ Actual Summary:     Mild opacity suggesting infection.
🤖 Generated Summary:  left upper lobe opacity.Normal lungs, but opacity seen in left upper lobe. fibrosis seen in right upper lobe, fibrosis. edema.'at taboolainventoryQuantitysenalpmwiki The externalToEVA istgaditionalIn�)... Planned evenlyStreamerBotTHIS�

🔹 Report 2:
Mild nodule detected in the bilateral lungs, consistent with fibrosis.
✅ Actual Summary:     Normal lungs, but nodule seen in bilateral lungs.
🤖 Generated Summary:  bilateral lungs noduleNormal lungs, but nodule seen in bilateral lungs, bilateral lungs.Normal lungs. fibrosis suggesting fibrosis. infiltrate. pneumonia.  'inventoryQuantitypmwiki attaboolasenal The externalToEVA.</In the istgaditional)... evenly��

🔹 Report 3:
There is a right lower lobe pleural effusion with air bronchograms.
✅ Actual Summary:     Mild pleural effusion suggesting atelectasis.
🤖 Generated Summary:  Normal lungs, bu

In [1]:
!pip install bert_score
!pip install nltk

  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached pyparsing-3.2.3-py3-none-any.whl.metadata (5.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 11.4 MB/s eta 0:00:00a 0:00:01
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 11.3 MB/s eta 0:00:00a 0:00:01
Using cached pyparsing-3.2.3-py3-none-any.whl (111 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [bert_score]7 [matplotlib]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 10.2 MB/s eta 0:00:00


In [3]:
from bert_score import score
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

references = [
    "Mild opacity suggesting infection.",
    "Normal lungs, but nodule seen in bilateral lungs.",
    "Mild pleural effusion suggesting atelectasis.",
    "Mild pleural effusion suggesting atelectasis.",
    "Normal lungs, but pleural effusion seen in cardiac silhouette."
]

candidates = [
    "left upper lobe opacity. Normal lungs, but opacity seen in left upper lobe.",
    "bilateral lungs nodule. Normal lungs, but nodule seen in bilateral lungs.",
    "Normal lungs, but pleural effusion seen in right lower lobe.",
    "Normal lungs, but pleural effusion seen in bilateral lungs.",
    "Normal lungs, but pleural effusion seen in cardiac silhouette."
]

# --- BERTScore ---
P, R, F1 = score(candidates, references, lang="en", verbose=True)
print("\nAverage BERTScore F1:", F1.mean().item())

# --- BLEU Score ---
print("\nBLEU Scores:")
for ref, cand in zip(references, candidates):
    ref_tokens = ref.split()
    cand_tokens = cand.split()
    bleu = sentence_bleu([ref_tokens], cand_tokens, smoothing_function=SmoothingFunction().method1)
    print(f"BLEU: {bleu:.4f}")


/opt/anaconda3/envs/rad-nlp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/anaconda3/envs/rad-nlp/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
W0617 22:28:24.000000 1210 site-packages/torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
/opt/anaconda3/envs/rad-nlp/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_downlo

calculating scores...
computing bert embedding.


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  6.47it/s]


computing greedy matching.


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 87.24it/s]


done in 0.17 seconds, 29.81 sentences/sec

Average BERTScore F1: 0.9285193681716919

BLEU Scores:
BLEU: 0.0155
BLEU: 0.6787
BLEU: 0.0446
BLEU: 0.0507
BLEU: 1.0000
